# 1. Setting up

## 1.1 Install the client library

In [1]:
# pip install azure-cognitiveservices-vision-customvision

## 1.2 Create a new python application

In [2]:
import json
  
# Open JSON file
with open('/Users/davide.cas/Downloads/COCO Dataset/annotations/instances_val2017.json') as json_file:
    data = json.load(json_file)
    
data.keys()

dict_keys(['info', 'licenses', 'images', 'annotations', 'categories'])

In [13]:
data["categories"]

[{'supercategory': 'person', 'id': 1, 'name': 'person'},
 {'supercategory': 'vehicle', 'id': 2, 'name': 'bicycle'},
 {'supercategory': 'vehicle', 'id': 3, 'name': 'car'},
 {'supercategory': 'vehicle', 'id': 4, 'name': 'motorcycle'},
 {'supercategory': 'vehicle', 'id': 5, 'name': 'airplane'},
 {'supercategory': 'vehicle', 'id': 6, 'name': 'bus'},
 {'supercategory': 'vehicle', 'id': 7, 'name': 'train'},
 {'supercategory': 'vehicle', 'id': 8, 'name': 'truck'},
 {'supercategory': 'vehicle', 'id': 9, 'name': 'boat'},
 {'supercategory': 'outdoor', 'id': 10, 'name': 'traffic light'},
 {'supercategory': 'outdoor', 'id': 11, 'name': 'fire hydrant'},
 {'supercategory': 'outdoor', 'id': 13, 'name': 'stop sign'},
 {'supercategory': 'outdoor', 'id': 14, 'name': 'parking meter'},
 {'supercategory': 'outdoor', 'id': 15, 'name': 'bench'},
 {'supercategory': 'animal', 'id': 16, 'name': 'bird'},
 {'supercategory': 'animal', 'id': 17, 'name': 'cat'},
 {'supercategory': 'animal', 'id': 18, 'name': 'dog'},

# 2. Training
## 2.1 Classes to be trained

Remote, mouse, laptop, keyboard, tv, microwave, sink, oven, toaster, knife

In [4]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
from PIL import Image
from skimage import io
import os, time, uuid
import urllib.request
import requests
import numpy as np

# Replace with valid values
ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com/"
training_key = "aced45df217b40c29c887a160d609ff4"
prediction_key = "aced45df217b40c29c887a160d609ff4"
prediction_resource_id = "/subscriptions/51a91f45-a201-4329-947c-d46fb0565938/resourceGroups/davidecas/providers/Microsoft.CognitiveServices/accounts/davidecas"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

publish_iteration_name = "detectModel"

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection" and domain.name == "General")

# Create a new project
print ("Creating project...")
# Use uuid to avoid project name collisions.
project = trainer.create_project(str(uuid.uuid4()), domain_id=obj_detection_domain.id)

###

n_categories = len(data["categories"])
category_ids = []
sel_categories = [
    "remote",
    "mouse",
    "laptop",
    "keyboard",
    "tv",
    "microwave",
    "sink",
    "toaster",
    "knife",
    "cell phone"
]

for i in range(len(sel_categories)):
    for j in range(n_categories):
        if sel_categories[i] == data["categories"][j]["name"]:
            category_ids.append(data["categories"][j]["id"])
            break

###

n_entries = len(data["annotations"])
total_length = len(sel_categories)
progress = total_length

for x in range(len(category_ids)):
    
    # actual progress percentage
    progress -= 1
    actual_progress = str(100 - (progress * 100 / total_length))
    print("[" + actual_progress + "%] Adding class:", sel_categories[x])
    
    
    image_ids = []
    for j in range(n_entries):
        category_id = data["annotations"][j]["category_id"]

        if category_id == category_ids[x]:
            image_id = data["annotations"][j]["image_id"]
            image_ids.append(image_id)

    ###

    image_ids = np.unique(image_ids)
    image_indexes = [j for j, y in enumerate(data["images"]) if y["id"] in image_ids]

    ###

    urls = []
    bbox = []
    imgs = []

    for image_index in image_indexes:
        urls.append(data["images"][image_index]["coco_url"])
        image_id = data["images"][image_index]["id"]

        annotations = [y["bbox"] for y in data["annotations"] if (y["image_id"] == image_id) and (y["category_id"] == category_ids[x])]
        bbox.append(annotations)
        imgs.append(data["images"][image_index])

    # Make two tags in the new project
    new_tag = trainer.create_tag(project.id, sel_categories[x])

    # Go through the data table above and create the images
    print ("Uploading images...")

    tagged_images_with_regions = []

    for i in range(len(bbox)):
        bboxes = bbox[i]
        regions = []

        for box in bboxes:

            x,y,w,h = box

            x /= imgs[i]["width"]
            y /= imgs[i]["height"]
            w /= imgs[i]["width"]
            h /= imgs[i]["height"]

            regions.append(Region(tag_id=new_tag.id, left=x,top=y,width=w,height=h))

        tagged_images_with_regions.append(ImageFileCreateEntry(name=i, contents=requests.get(urls[i], stream=True).content, regions=regions))

        if len(tagged_images_with_regions) == 64:
            upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
            if not upload_result.is_batch_successful: exit(-1)

            tagged_images_with_regions = []

    if len(tagged_images_with_regions) > 0:
        upload_result = trainer.create_images_from_files(project.id, ImageFileCreateBatch(images=tagged_images_with_regions))
        if not upload_result.is_batch_successful: exit(-1)

        tagged_images_with_regions = []

print ("Finished.")

Creating project...
[10.0%] Adding class: remote
Uploading images...
[20.0%] Adding class: mouse
Uploading images...
[30.0%] Adding class: laptop
Uploading images...
[40.0%] Adding class: keyboard
Uploading images...
[50.0%] Adding class: tv
Uploading images...
[60.0%] Adding class: microwave
Uploading images...
[70.0%] Adding class: sink
Uploading images...
[80.0%] Adding class: toaster
Uploading images...
[90.0%] Adding class: knife
Uploading images...
[100.0%] Adding class: cell phone
Uploading images...
Finished.


1159 photo uploaded.